<a href="https://colab.research.google.com/github/gauravraidata/IITJ-projects/blob/main/GPU_ASS_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import argparse, time, os, math, sys
from typing import Dict, List, Optional
import numpy as np
import pandas as pd
from scipy import sparse
import matplotlib.pyplot as plt

import cupy as cp
import cupyx.scipy.sparse as cpx_sparse

In [30]:
def load_item_session(csv_path: str, country: str = None):
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"CSV not found: {csv_path}")

    df = pd.read_csv(csv_path, encoding="ISO-8859-1")
    required = ["InvoiceNo", "StockCode", "Description", "Quantity", "UnitPrice"]
    for c in required:
        if c not in df.columns:
            raise RuntimeError(f"Required column '{c}' not found in CSV")

    df = df.dropna(subset=required)
    df = df[~df["InvoiceNo"].astype(str).str.startswith("C")]
    df = df[df["Quantity"] > 0]
    df = df[df["UnitPrice"] > 0.0]
    if country and "Country" in df.columns:
        df = df[df["Country"] == country]

    df["StockCode"] = df["StockCode"].astype(str)
    unique_products = pd.Series(df["StockCode"].unique())
    prod_to_idx = {p: i for i, p in enumerate(unique_products)}
    df["item_idx"] = df["StockCode"].map(prod_to_idx)

    df["InvoiceNo"] = df["InvoiceNo"].astype(str)
    unique_invoices = pd.Series(df["InvoiceNo"].unique())
    inv_to_idx = {inv: i for i, inv in enumerate(unique_invoices)}
    df["session_idx"] = df["InvoiceNo"].map(inv_to_idx)

    df_unique = df.drop_duplicates(subset=["session_idx", "item_idx"])

    rows = df_unique["item_idx"].to_numpy(dtype=np.int32)
    cols = df_unique["session_idx"].to_numpy(dtype=np.int32)
    data = (np.ones_like(rows, dtype=np.int8)).astype(np.int8)

    num_items = len(unique_products)
    num_sessions = len(unique_invoices)

    item_session = sparse.csr_matrix((data, (rows, cols)), shape=(num_items, num_sessions), dtype=np.int32)

    item_meta: Dict[int, Dict] = {}
    for idx, sub in df.groupby("item_idx"):
        desc = sub["Description"].mode().iloc[0] if "Description" in sub else ""
        item_meta[int(idx)] = {"stock_code": sub["StockCode"].iloc[0], "description": str(desc)}

    return item_session, item_meta, num_sessions

In [31]:
def scipy_csr_to_cupy_csr(scipy_csr: sparse.csr_matrix):
    coo = scipy_csr.tocoo()
    data_cp = cp.asarray(coo.data, dtype=cp.float32)
    row_cp = cp.asarray(coo.row)
    col_cp = cp.asarray(coo.col)
    cupy_coo = cpx_sparse.coo_matrix((data_cp, (row_cp, col_cp)), shape=scipy_csr.shape)
    return cupy_coo.tocsr()

In [32]:

def gpu_topk_all_items(item_session_cupy, k: int, batch_size: int = 64):
    num_items = int(item_session_cupy.shape[0])

    # diag: sessions containing item (on device)
    diag_gpu = item_session_cupy.multiply(item_session_cupy).sum(axis=1).ravel().astype(cp.int32)
    diag_host = cp.asnumpy(diag_gpu)

    topk_idx_list = [None] * num_items
    topk_scores_list = [None] * num_items

    start_time = time.time()
    sqrt_diag_gpu = cp.sqrt(cp.maximum(diag_gpu.astype(cp.float32), 1.0))
    anchors = np.arange(num_items, dtype=np.int32)

    for bstart in range(0, num_items, batch_size):
        bend = min(bstart + batch_size, num_items)
        batch_anchors = anchors[bstart:bend]
        B = len(batch_anchors)

        # slice rows and compute co-occurrence for the batch
        batch_rows_sp = item_session_cupy[batch_anchors, :]
        co_batch_sp = batch_rows_sp.dot(item_session_cupy.T)
        co_batch_dense = co_batch_sp.toarray().astype(cp.float32)  # (B, N)

        denom_matrix = sqrt_diag_gpu[cp.asarray(batch_anchors, dtype=cp.int32)].astype(cp.float32)[:, None] * sqrt_diag_gpu[None, :]

        sim_batch = cp.where(denom_matrix > 0.0, co_batch_dense / denom_matrix, 0.0).astype(cp.float32)

        # vectorized zero-out self positions (keep entirely on device)
        sim_batch[cp.arange(B, dtype=cp.int32), cp.asarray(batch_anchors, dtype=cp.int32)] = 0.0

        # per-row top-k (device)
        if k >= num_items:
            topk_idx_batch = cp.argsort(-sim_batch, axis=1)[:, :k]
            topk_scores_batch = cp.take_along_axis(sim_batch, topk_idx_batch, axis=1)
        else:
            part_idx = cp.argpartition(-sim_batch, k, axis=1)[:, :k]
            part_scores = cp.take_along_axis(sim_batch, part_idx, axis=1)
            order = cp.argsort(-part_scores, axis=1)
            topk_idx_batch = cp.take_along_axis(part_idx, order, axis=1)
            topk_scores_batch = cp.take_along_axis(part_scores, order, axis=1)

        # Transfer small B x k arrays to host
        topk_idx_batch_host = cp.asnumpy(topk_idx_batch.astype(cp.int32))
        topk_scores_batch_host = cp.asnumpy(topk_scores_batch.astype(cp.float32))

        for local_r in range(B):
            global_i = int(batch_anchors[local_r])
            topk_idx_list[global_i] = topk_idx_batch_host[local_r].astype(np.int32)
            topk_scores_list[global_i] = topk_scores_batch_host[local_r].astype(np.float32)

        elapsed = time.time() - start_time
        print(f"[GPU batch] anchors {bstart}-{bend-1} processed (B={B}), elapsed={elapsed:.2f}s")

    gpu_elapsed = time.time() - start_time
    return topk_idx_list, topk_scores_list, diag_host, gpu_elapsed

In [33]:
def cpu_topk_all_items(item_session_scipy, k: int):
    num_items = item_session_scipy.shape[0]
    diag = item_session_scipy.multiply(item_session_scipy).sum(axis=1).A1.astype(np.int32)
    topk_idx_list = [None] * num_items
    topk_scores_list = [None] * num_items

    start_time = time.time()
    for i in range(num_items):
        row = item_session_scipy.getrow(i)
        co_row_sp = row.dot(item_session_scipy.T)
        co_row = co_row_sp.toarray().ravel().astype(np.float32)

        denom = math.sqrt(diag[i]) * np.sqrt(np.maximum(diag, 1.0))
        with np.errstate(divide='ignore', invalid='ignore'):
            sim_row = np.where(denom > 0, co_row / denom, 0.0).astype(np.float32)
        sim_row[i] = 0.0

        if k >= sim_row.size:
            idx = np.argsort(-sim_row)[:k]
        else:
            part = np.argpartition(-sim_row, k)[:k]
            idx = part[np.argsort(-sim_row[part])]

        topk_idx_list[i] = idx.astype(np.int32)
        topk_scores_list[i] = sim_row[idx].astype(np.float32)

        if (i + 1) % 500 == 0 or (i + 1) == num_items:
            elapsed = time.time() - start_time
            print(f"[CPU] completed {i+1}/{num_items} items, elapsed={elapsed:.2f}s")

    cpu_elapsed = time.time() - start_time
    return topk_idx_list, topk_scores_list, diag, cpu_elapsed


In [34]:
def build_anchor_table(anchor_idx: int, topk_idx_list: List[np.ndarray], topk_scores_list: List[np.ndarray],
                       item_session_scipy: sparse.csr_matrix, diag: np.ndarray, item_meta: Dict[int, Dict],
                       num_sessions: int) -> pd.DataFrame:
    co_row_sp = item_session_scipy.getrow(anchor_idx).dot(item_session_scipy.T)
    co_row = co_row_sp.toarray().ravel().astype(np.int32)
    denom = math.sqrt(diag[anchor_idx]) * np.sqrt(np.maximum(diag, 1.0))
    with np.errstate(divide='ignore', invalid='ignore'):
        sim_exact = np.where(denom > 0.0, co_row.astype(np.float32) / denom, 0.0).astype(np.float32)
    sim_exact[anchor_idx] = 0.0

    top_idx = topk_idx_list[anchor_idx]
    rows = []
    for rank, other in enumerate(top_idx, start=1):
        co_cnt = int(co_row[int(other)])
        sim_score = float(sim_exact[int(other)])
        lift = 0.0
        if diag[anchor_idx] > 0 and diag[int(other)] > 0:
            p_ab = co_cnt / float(num_sessions)
            p_a = diag[anchor_idx] / float(num_sessions)
            p_b = diag[int(other)] / float(num_sessions)
            lift = p_ab / (p_a * p_b) if (p_a * p_b) > 0 else 0.0
        rows.append({
            "rank": rank,
            "item_index": int(other),
            "stock_code": item_meta[int(other)]["stock_code"] if int(other) in item_meta else "",
            "description": item_meta[int(other)]["description"] if int(other) in item_meta else "",
            "cooccurrence": co_cnt,
            "similarity": sim_score,
            "lift": lift,
            "sessions_with_other": int(diag[int(other)])
        })
    return pd.DataFrame(rows)

In [35]:

def plot_timing_inline(cpu_time, gpu_transfer, gpu_compute):
    gpu_total = gpu_transfer + gpu_compute
    labels = ["CPU (SciPy) per-row", "GPU transfer", "GPU compute", "GPU total"]
    times = [cpu_time, gpu_transfer, gpu_compute, gpu_total]
    plt.figure(figsize=(8,4))
    bars = plt.bar(labels, times)
    plt.ylabel("Seconds")
    plt.title("CPU vs GPU timing breakdown")
    for bar in bars:
        h = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, h + 0.3, f"{h:.2f}s", ha='center', va='bottom', fontsize=9)
    plt.tight_layout()
    plt.show()

In [36]:
def save_anchor_tables_and_csvs(anchor_idx, recs_anchor_gpu, recs_anchor_cpu, save_prefix="anchor_recs"):
    gpu_csv = f"{save_prefix}_gpu.csv"
    cpu_csv = f"{save_prefix}_cpu.csv"
    recs_anchor_gpu.to_csv(gpu_csv, index=False)
    recs_anchor_cpu.to_csv(cpu_csv, index=False)
    print(f"Saved anchor tables: {gpu_csv}, {cpu_csv}")

In [37]:
def print_pipeline_explanation():
    print("\n=== GPU pipeline explanation & summary ===")
    print("1) Data ingestion & preprocessing (host): CSV -> pandas -> SciPy CSR.")
    print("2) Transfer to GPU: SciPy CSR -> CuPy CSR (single conversion).")
    print("3) Batched GPU compute:")
    print("   - For each batch: compute BxN cooccurrence, normalize (device), top-k (device), transfer BxK back.")
    print("4) CPU baseline: per-row sparse matmul + numpy top-k (highly optimized on CPU).")
    print("\nWhy GPU can be slower than CPU sometimes: kernel-launch & allocation overhead for many tiny tasks.")
    print("Tune batch_size (e.g., 32/64/128) to amortize overhead; larger B uses more GPU memory but fewer launches.")
    print("==========================================\n")

In [38]:
def main(argv=None):
    parser = argparse.ArgumentParser()
    parser.add_argument("--csv", type=str, default="OnlineRetail_cached.csv", help="Path to local CSV")
    parser.add_argument("--k", type=int, default=10, help="Top-K")
    parser.add_argument("--batch_size", type=int, default=64, help="GPU batch size for anchors")
    parser.add_argument("--anchor_by", choices=["most_popular","random","id"], default="most_popular")
    parser.add_argument("--anchor_id", type=int, default=None)
    parser.add_argument("--country", type=str, default=None)
    parser.add_argument("--save_all", action="store_true", help="Save full recs CSVs")
    parser.add_argument("--inline_plot", action="store_true", help="Show timing plot inline (not saved)")
    args = parser.parse_args(argv)

    csv_path = args.csv
    k = args.k
    batch_size = args.batch_size

    print("STEP 1: Load dataset and build host CSR (SciPy).")
    t0 = time.time()
    item_session_scipy, item_meta, num_sessions = load_item_session(csv_path, country=args.country)
    t1 = time.time()
    print(f" loaded: items={item_session_scipy.shape[0]}, sessions={item_session_scipy.shape[1]}, time={t1-t0:.2f}s")

    print("\nSTEP 2: Transfer CSR to GPU (CuPy).")
    t0 = time.time()
    item_session_gpu = scipy_csr_to_cupy_csr(item_session_scipy)
    cp.cuda.Stream.null.synchronize()
    t_transfer = time.time() - t0
    print(f" transfer time: {t_transfer:.2f}s")

    print("\nSTEP 3: GPU compute top-K per item (batched).")
    t0 = time.time()
    topk_idx_gpu, topk_scores_gpu, diag_gpu, t_gpu_compute = gpu_topk_all_items(item_session_gpu, k=k, batch_size=batch_size)
    t_gpu_total = t_transfer + t_gpu_compute
    print(f" GPU total (transfer+compute): {t_gpu_total:.2f}s (compute only: {t_gpu_compute:.2f}s)")

    print("\nSTEP 4: CPU baseline top-K per item (for comparison).")
    topk_idx_cpu, topk_scores_cpu, diag_cpu, t_cpu = cpu_topk_all_items(item_session_scipy, k=k)
    print(f" CPU total compute: {t_cpu:.2f}s")

    # choose anchor
    if args.anchor_by == "most_popular":
        anchor = int(np.argmax(diag_cpu))
    elif args.anchor_by == "random":
        anchor = int(np.random.default_rng(1).integers(0, item_session_scipy.shape[0]))
    else:
        if args.anchor_id is None:
            raise ValueError("anchor_id required when anchor_by=='id'")
        anchor = args.anchor_id

    anchor_stock = item_meta[anchor]['stock_code']
    anchor_name  = item_meta[anchor]['description']

    print(f"\nAnchor item index = {anchor}; "
          f"stock_code = {anchor_stock}; "
          f"description = {anchor_name}; "
          f"sessions_with_anchor = {int(diag_cpu[anchor])}")

    # build anchor tables
    recs_anchor_gpu = build_anchor_table(anchor, topk_idx_gpu, topk_scores_gpu, item_session_scipy, diag_gpu, item_meta, num_sessions)
    recs_anchor_cpu = build_anchor_table(anchor, topk_idx_cpu, topk_scores_cpu, item_session_scipy, diag_cpu, item_meta, num_sessions)

    print("\nTop-K (GPU-derived):")
    print(recs_anchor_gpu.to_string(index=False))

    print("\nTop-K (CPU-derived):")
    print(recs_anchor_cpu.to_string(index=False))

    # Example recommendations for 2 products (top-2 by popularity)
    try:
        top2 = list(np.argsort(-diag_cpu)[:2])
        print(f"\n\n=== Recommendation examples for 2 products (indexes): {top2} ===")
        for ex_anchor in top2:
            ex_stock = item_meta[ex_anchor]['stock_code']
            ex_name = item_meta[ex_anchor]['description']
            print(f"\nExample Anchor {ex_anchor} (stock_code = {ex_stock}, description = {ex_name}, sessions = {int(diag_cpu[ex_anchor])})")
            ex_recs_gpu = build_anchor_table(ex_anchor, topk_idx_gpu, topk_scores_gpu, item_session_scipy, diag_gpu, item_meta, num_sessions)
            ex_recs_cpu = build_anchor_table(ex_anchor, topk_idx_cpu, topk_scores_cpu, item_session_scipy, diag_cpu, item_meta, num_sessions)
            print("\nGPU-derived top-5:")
            print(ex_recs_gpu.head(5).to_string(index=False))
            print("\nCPU-derived top-5:")
            print(ex_recs_cpu.head(5).to_string(index=False))
    except Exception as e:
        print("Could not produce 2-product examples:", e)

    # Save anchor tables and optionally full recs
    save_anchor_tables_and_csvs(anchor, recs_anchor_gpu, recs_anchor_cpu, save_prefix=f"anchor_{anchor}_recs")
    if args.save_all:
        print("\nSaving full recommendation CSVs (CPU & GPU). This may be large.")
        # CPU
        rows = []
        for i in range(len(topk_idx_cpu)):
            idxs = topk_idx_cpu[i]
            scores = topk_scores_cpu[i]
            for r,(other,score) in enumerate(zip(idxs,scores), start=1):
                rows.append({
                    "item_index": i,
                    "stock_code_item": item_meta[i]["stock_code"],
                    "item_description": item_meta[i]["description"],
                    "rank": r,
                    "other_index": int(other),
                    "other_stock_code": item_meta[int(other)]["stock_code"],
                    "other_description": item_meta[int(other)]["description"],
                    "similarity": float(score),
                })
        pd.DataFrame(rows).to_csv("recs_all_cpu.csv", index=False)
        # GPU
        rows = []
        for i in range(len(topk_idx_gpu)):
            idxs = topk_idx_gpu[i]
            scores = topk_scores_gpu[i]
            for r,(other,score) in enumerate(zip(idxs,scores), start=1):
                rows.append({
                    "item_index": i,
                    "stock_code_item": item_meta[i]["stock_code"],
                    "item_description": item_meta[i]["description"],
                    "rank": r,
                    "other_index": int(other),
                    "other_stock_code": item_meta[int(other)]["stock_code"],
                    "other_description": item_meta[int(other)]["description"],
                    "similarity": float(score),
                })
        pd.DataFrame(rows).to_csv("recs_all_gpu.csv", index=False)
        print("Saved recs_all_cpu.csv and recs_all_gpu.csv")

    # Summary table & plot
    print("\nSUMMARY:")
    df_summary = pd.DataFrame([
        {"engine": "CPU (SciPy) per-row", "time_sec": round(t_cpu,3)},
        {"engine": "GPU (CuPy) transfer", "time_sec": round(t_transfer,3)},
        {"engine": "GPU (CuPy) compute", "time_sec": round(t_gpu_compute,3)},
        {"engine": "GPU (CuPy) total", "time_sec": round(t_gpu_total,3)}
    ])
    speedup = (t_cpu / t_gpu_total) if (t_gpu_total > 0) else float('nan')
    df_summary_print = df_summary.copy()
    if not np.isnan(speedup):
        df_summary_print.loc[len(df_summary_print)] = {"engine": "CPU_time / GPU_total_speedup", "time_sec": round(speedup,3)}
    print(df_summary_print.to_string(index=False))

    # Inline plot or saved plot based on flag
    if args.inline_plot:
        try:
            plot_timing_inline(t_cpu, t_transfer, t_gpu_compute)
        except Exception as e:
            print("Warning: inline plot failed:", e)
    else:
        try:
            # fallback: save plot file
            from matplotlib import pyplot as pltfile
            gpu_total = t_transfer + t_gpu_compute
            df_plot = pd.DataFrame([
                {"engine": "CPU (SciPy) per-row", "time_sec": t_cpu},
                {"engine": "GPU (CuPy) transfer", "time_sec": t_transfer},
                {"engine": "GPU (CuPy) compute", "time_sec": t_gpu_compute},
                {"engine": "GPU (CuPy) total", "time_sec": gpu_total}
            ])
            pltfile.figure(figsize=(8,4))
            bars = pltfile.bar(df_plot["engine"], df_plot["time_sec"])
            pltfile.ylabel("Seconds")
            pltfile.title("CPU vs GPU timing breakdown")
            for bar in bars:
                h = bar.get_height()
                pltfile.text(bar.get_x() + bar.get_width()/2, h + 0.5, f"{h:.2f}s", ha='center', va='bottom', fontsize=9)
            pltfile.tight_layout()
            pltfile.savefig("timing_comparison.png", dpi=150)
            pltfile.close()
            print("Saved timing comparison plot to timing_comparison.png")
        except Exception as e:
            print("Warning: plot creation failed:", e)

    print_pipeline_explanation()
    print("\nDone.")

if __name__ == '__main__':
    if hasattr(sys, 'argv') and len(sys.argv) > 0 and \
       (sys.argv[0].endswith('ipykernel_launcher.py') or sys.argv[0].endswith('colab_kernel_launcher.py')):
        main(argv=[])
    else:
        main(argv=sys.argv[1:])


STEP 1: Load dataset and build host CSR (SciPy).
 loaded: items=3922, sessions=19960, time=2.48s

STEP 2: Transfer CSR to GPU (CuPy).
 transfer time: 0.02s

STEP 3: GPU compute top-K per item (batched).
[GPU batch] anchors 0-63 processed (B=64), elapsed=0.02s
[GPU batch] anchors 64-127 processed (B=64), elapsed=0.04s
[GPU batch] anchors 128-191 processed (B=64), elapsed=0.05s
[GPU batch] anchors 192-255 processed (B=64), elapsed=0.07s
[GPU batch] anchors 256-319 processed (B=64), elapsed=0.09s
[GPU batch] anchors 320-383 processed (B=64), elapsed=0.10s
[GPU batch] anchors 384-447 processed (B=64), elapsed=0.12s
[GPU batch] anchors 448-511 processed (B=64), elapsed=0.14s
[GPU batch] anchors 512-575 processed (B=64), elapsed=0.15s
[GPU batch] anchors 576-639 processed (B=64), elapsed=0.17s
[GPU batch] anchors 640-703 processed (B=64), elapsed=0.19s
[GPU batch] anchors 704-767 processed (B=64), elapsed=0.21s
[GPU batch] anchors 768-831 processed (B=64), elapsed=0.23s
[GPU batch] anchors 8

In [39]:
import argparse, time, os, math, sys
from typing import Dict, List, Optional
import numpy as np
import pandas as pd
from scipy import sparse
import matplotlib.pyplot as plt

import cupy as cp
import cupyx.scipy.sparse as cpx_sparse

def parse_transaction_data(file_path: str, region: str = None):
    """Parse retail transaction data from CSV file"""
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File does not exist: {file_path}")

    transactions_df = pd.read_csv(file_path, encoding="ISO-8859-1")
    mandatory_fields = ["InvoiceNo", "StockCode", "Description", "Quantity", "UnitPrice"]
    for field in mandatory_fields:
        if field not in transactions_df.columns:
            raise RuntimeError(f"Missing mandatory field '{field}' in dataset")

    # Data cleaning and filtering pipeline
    transactions_df = transactions_df.dropna(subset=mandatory_fields)
    transactions_df = transactions_df[~transactions_df["InvoiceNo"].astype(str).str.startswith("C")]
    transactions_df = transactions_df[transactions_df["Quantity"] > 0]
    transactions_df = transactions_df[transactions_df["UnitPrice"] > 0.0]
    if region and "Country" in transactions_df.columns:
        transactions_df = transactions_df[transactions_df["Country"] == region]

    # Product encoding
    transactions_df["StockCode"] = transactions_df["StockCode"].astype(str)
    distinct_products = pd.Series(transactions_df["StockCode"].unique())
    product_mapping = {prod: position for position, prod in enumerate(distinct_products)}
    transactions_df["product_id"] = transactions_df["StockCode"].map(product_mapping)

    # Invoice/session encoding
    transactions_df["InvoiceNo"] = transactions_df["InvoiceNo"].astype(str)
    distinct_invoices = pd.Series(transactions_df["InvoiceNo"].unique())
    invoice_mapping = {invoice: position for position, invoice in enumerate(distinct_invoices)}
    transactions_df["basket_id"] = transactions_df["InvoiceNo"].map(invoice_mapping)

    # Remove duplicate product-basket pairs
    unique_pairs = transactions_df.drop_duplicates(subset=["basket_id", "product_id"])

    # Build sparse matrix representation
    row_indices = unique_pairs["product_id"].to_numpy(dtype=np.int32)
    col_indices = unique_pairs["basket_id"].to_numpy(dtype=np.int32)
    values = (np.ones_like(row_indices, dtype=np.int8)).astype(np.int8)

    total_products = len(distinct_products)
    total_baskets = len(distinct_invoices)

    product_basket_matrix = sparse.csr_matrix((values, (row_indices, col_indices)),
                                              shape=(total_products, total_baskets),
                                              dtype=np.int32)

    # Extract product metadata
    product_metadata: Dict[int, Dict] = {}
    for product_idx, group_data in transactions_df.groupby("product_id"):
        desc_value = group_data["Description"].mode().iloc[0] if "Description" in group_data else ""
        product_metadata[int(product_idx)] = {
            "stock_code": group_data["StockCode"].iloc[0],
            "description": str(desc_value)
        }

    return product_basket_matrix, product_metadata, total_baskets

def convert_scipy_to_cupy_sparse(scipy_matrix: sparse.csr_matrix):
    """Transfer sparse matrix from CPU (SciPy) to GPU (CuPy)"""
    coo_format = scipy_matrix.tocoo()
    gpu_data = cp.asarray(coo_format.data, dtype=cp.float32)
    gpu_rows = cp.asarray(coo_format.row)
    gpu_cols = cp.asarray(coo_format.col)
    gpu_coo_matrix = cpx_sparse.coo_matrix((gpu_data, (gpu_rows, gpu_cols)), shape=scipy_matrix.shape)
    return gpu_coo_matrix.tocsr()


def compute_similarities_gpu_batched(product_matrix_gpu, top_k: int, batch_count: int = 64):
    """Compute top-K similar products using GPU acceleration with batching"""
    total_products = int(product_matrix_gpu.shape[0])

    # Calculate product frequency (diagonal elements)
    diagonal_values_gpu = product_matrix_gpu.multiply(product_matrix_gpu).sum(axis=1).ravel().astype(cp.int32)
    diagonal_values_cpu = cp.asnumpy(diagonal_values_gpu)

    top_indices_collection = [None] * total_products
    top_scores_collection = [None] * total_products

    computation_start = time.time()
    sqrt_diagonal_gpu = cp.sqrt(cp.maximum(diagonal_values_gpu.astype(cp.float32), 1.0))
    product_indices = np.arange(total_products, dtype=np.int32)

    for batch_start in range(0, total_products, batch_count):
        batch_end = min(batch_start + batch_count, total_products)
        current_batch = product_indices[batch_start:batch_end]
        batch_length = len(current_batch)

        # Extract batch rows and compute cooccurrence matrix
        batch_matrix = product_matrix_gpu[current_batch, :]
        cooccurrence_batch = batch_matrix.dot(product_matrix_gpu.T)
        cooccurrence_dense = cooccurrence_batch.toarray().astype(cp.float32)

        denominator_matrix = sqrt_diagonal_gpu[cp.asarray(current_batch, dtype=cp.int32)].astype(cp.float32)[:, None] * sqrt_diagonal_gpu[None, :]

        similarity_batch = cp.where(denominator_matrix > 0.0,
                                    cooccurrence_dense / denominator_matrix,
                                    0.0).astype(cp.float32)

        # Zero out self-similarities
        similarity_batch[cp.arange(batch_length, dtype=cp.int32),
                        cp.asarray(current_batch, dtype=cp.int32)] = 0.0

        # Extract top-K per row
        if top_k >= total_products:
            top_k_indices = cp.argsort(-similarity_batch, axis=1)[:, :top_k]
            top_k_scores = cp.take_along_axis(similarity_batch, top_k_indices, axis=1)
        else:
            partition_indices = cp.argpartition(-similarity_batch, top_k, axis=1)[:, :top_k]
            partition_scores = cp.take_along_axis(similarity_batch, partition_indices, axis=1)
            sort_order = cp.argsort(-partition_scores, axis=1)
            top_k_indices = cp.take_along_axis(partition_indices, sort_order, axis=1)
            top_k_scores = cp.take_along_axis(partition_scores, sort_order, axis=1)

        # Transfer results to CPU
        top_k_indices_cpu = cp.asnumpy(top_k_indices.astype(cp.int32))
        top_k_scores_cpu = cp.asnumpy(top_k_scores.astype(cp.float32))

        for local_index in range(batch_length):
            global_product = int(current_batch[local_index])
            top_indices_collection[global_product] = top_k_indices_cpu[local_index].astype(np.int32)
            top_scores_collection[global_product] = top_k_scores_cpu[local_index].astype(np.float32)

        elapsed_time = time.time() - computation_start
        print(f"[GPU processing] batch {batch_start}-{batch_end-1} complete (size={batch_length}), time={elapsed_time:.2f}s")

    total_gpu_time = time.time() - computation_start
    return top_indices_collection, top_scores_collection, diagonal_values_cpu, total_gpu_time

def compute_similarities_cpu_baseline(product_matrix_scipy, top_k: int):
    """Baseline CPU implementation for similarity computation"""
    total_products = product_matrix_scipy.shape[0]
    diagonal_values = product_matrix_scipy.multiply(product_matrix_scipy).sum(axis=1).A1.astype(np.int32)
    top_indices_collection = [None] * total_products
    top_scores_collection = [None] * total_products

    computation_start = time.time()
    for product_id in range(total_products):
        product_vector = product_matrix_scipy.getrow(product_id)
        cooccurrence_vector = product_vector.dot(product_matrix_scipy.T)
        cooccurrence_array = cooccurrence_vector.toarray().ravel().astype(np.float32)

        denominator = math.sqrt(diagonal_values[product_id]) * np.sqrt(np.maximum(diagonal_values, 1.0))
        with np.errstate(divide='ignore', invalid='ignore'):
            similarity_vector = np.where(denominator > 0,
                                        cooccurrence_array / denominator,
                                        0.0).astype(np.float32)
        similarity_vector[product_id] = 0.0

        if top_k >= similarity_vector.size:
            sorted_indices = np.argsort(-similarity_vector)[:top_k]
        else:
            partition = np.argpartition(-similarity_vector, top_k)[:top_k]
            sorted_indices = partition[np.argsort(-similarity_vector[partition])]

        top_indices_collection[product_id] = sorted_indices.astype(np.int32)
        top_scores_collection[product_id] = similarity_vector[sorted_indices].astype(np.float32)

        if (product_id + 1) % 500 == 0 or (product_id + 1) == total_products:
            elapsed_time = time.time() - computation_start
            print(f"[CPU processing] {product_id+1}/{total_products} products done, time={elapsed_time:.2f}s")

    total_cpu_time = time.time() - computation_start
    return top_indices_collection, top_scores_collection, diagonal_values, total_cpu_time


def generate_recommendation_report(reference_product: int, top_indices_list: List[np.ndarray],
                                  top_scores_list: List[np.ndarray],
                                  product_matrix_scipy: sparse.csr_matrix,
                                  diagonal_values: np.ndarray,
                                  product_metadata: Dict[int, Dict],
                                  total_baskets: int) -> pd.DataFrame:
    """Generate detailed recommendation report for a reference product"""
    cooccurrence_vector = product_matrix_scipy.getrow(reference_product).dot(product_matrix_scipy.T)
    cooccurrence_array = cooccurrence_vector.toarray().ravel().astype(np.int32)
    denominator = math.sqrt(diagonal_values[reference_product]) * np.sqrt(np.maximum(diagonal_values, 1.0))
    with np.errstate(divide='ignore', invalid='ignore'):
        similarity_exact = np.where(denominator > 0.0,
                                    cooccurrence_array.astype(np.float32) / denominator,
                                    0.0).astype(np.float32)
    similarity_exact[reference_product] = 0.0

    recommended_indices = top_indices_list[reference_product]
    report_rows = []
    for position, related_product in enumerate(recommended_indices, start=1):
        cooccurrence_count = int(cooccurrence_array[int(related_product)])
        similarity_score = float(similarity_exact[int(related_product)])
        lift_value = 0.0
        if diagonal_values[reference_product] > 0 and diagonal_values[int(related_product)] > 0:
            prob_both = cooccurrence_count / float(total_baskets)
            prob_reference = diagonal_values[reference_product] / float(total_baskets)
            prob_related = diagonal_values[int(related_product)] / float(total_baskets)
            lift_value = prob_both / (prob_reference * prob_related) if (prob_reference * prob_related) > 0 else 0.0
        report_rows.append({
            "rank": position,
            "item_index": int(related_product),
            "stock_code": product_metadata[int(related_product)]["stock_code"] if int(related_product) in product_metadata else "",
            "description": product_metadata[int(related_product)]["description"] if int(related_product) in product_metadata else "",
            "cooccurrence": cooccurrence_count,
            "similarity": similarity_score,
            "lift": lift_value,
            "sessions_with_other": int(diagonal_values[int(related_product)])
        })
    return pd.DataFrame(report_rows)


def visualize_performance_comparison(cpu_duration, gpu_transfer_time, gpu_compute_time):
    """Create visualization comparing CPU and GPU performance"""
    gpu_combined = gpu_transfer_time + gpu_compute_time
    category_labels = ["CPU (SciPy) computation", "GPU data transfer", "GPU computation", "GPU combined"]
    duration_values = [cpu_duration, gpu_transfer_time, gpu_compute_time, gpu_combined]
    plt.figure(figsize=(8,4))
    bar_chart = plt.bar(category_labels, duration_values)
    plt.ylabel("Execution Time (seconds)")
    plt.title("Performance Analysis: CPU vs GPU")
    for bar in bar_chart:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, height + 0.3,
                f"{height:.2f}s", ha='center', va='bottom', fontsize=9)
    plt.tight_layout()
    plt.show()

def export_recommendation_tables(reference_product, gpu_recommendations, cpu_recommendations, output_prefix="product_recs"):
    """Save recommendation tables to CSV files"""
    gpu_output = f"{output_prefix}_gpu.csv"
    cpu_output = f"{output_prefix}_cpu.csv"
    gpu_recommendations.to_csv(gpu_output, index=False)
    cpu_recommendations.to_csv(cpu_output, index=False)
    print(f"Exported recommendation tables: {gpu_output}, {cpu_output}")

def display_implementation_summary():
    """Display technical summary of the recommendation pipeline"""
    print("\n=== Implementation Architecture Overview ===")
    print("Step 1: Data preprocessing on host - CSV parsing into SciPy sparse matrix format.")
    print("Step 2: Data migration to GPU - Convert SciPy CSR to CuPy CSR (single transfer).")
    print("Step 3: GPU batch processing:")
    print("   - Process batches: calculate BxN cooccurrence, normalize on device, extract top-k, transfer results.")
    print("Step 4: CPU reference implementation: row-wise sparse operations with NumPy top-k selection.")
    print("\nPerformance considerations: GPU overhead from kernel launches may exceed benefits for small workloads.")
    print("Optimization strategy: Increase batch_size parameter (32/64/128) to reduce overhead; larger batches require more VRAM.")
    print("============================================\n")

def main(argv=None):
    argument_parser = argparse.ArgumentParser()
    argument_parser.add_argument("--csv", type=str, default="OnlineRetail_cached.csv", help="Input CSV file path")
    argument_parser.add_argument("--k", type=int, default=10, help="Number of top recommendations")
    argument_parser.add_argument("--batch_size", type=int, default=64, help="Batch size for GPU processing")
    argument_parser.add_argument("--anchor_by", choices=["most_popular","random","id"], default="most_popular")
    argument_parser.add_argument("--anchor_id", type=int, default=None)
    argument_parser.add_argument("--country", type=str, default=None)
    argument_parser.add_argument("--save_all", action="store_true", help="Export complete recommendation datasets")
    argument_parser.add_argument("--inline_plot", action="store_true", help="Display performance visualization inline")
    config = argument_parser.parse_args(argv)

    input_file = config.csv
    recommendations_count = config.k
    processing_batch_size = config.batch_size

    print("Phase 1: Loading and preprocessing transaction dataset.")
    start_timestamp = time.time()
    product_basket_matrix_cpu, product_metadata, total_baskets = parse_transaction_data(input_file, region=config.country)
    end_timestamp = time.time()
    print(f" Dataset loaded: products={product_basket_matrix_cpu.shape[0]}, baskets={product_basket_matrix_cpu.shape[1]}, duration={end_timestamp-start_timestamp:.2f}s")

    print("\nPhase 2: Transferring sparse matrix to GPU memory.")
    start_timestamp = time.time()
    product_basket_matrix_gpu = convert_scipy_to_cupy_sparse(product_basket_matrix_cpu)
    cp.cuda.Stream.null.synchronize()
    transfer_duration = time.time() - start_timestamp
    print(f" Transfer completed: {transfer_duration:.2f}s")

    print("\nPhase 3: Computing recommendations using GPU (batched processing).")
    start_timestamp = time.time()
    gpu_top_indices, gpu_top_scores, gpu_diagonal, gpu_compute_duration = compute_similarities_gpu_batched(
        product_basket_matrix_gpu, top_k=recommendations_count, batch_count=processing_batch_size)
    gpu_total_duration = transfer_duration + gpu_compute_duration
    print(f" GPU execution (transfer+compute): {gpu_total_duration:.2f}s (compute only: {gpu_compute_duration:.2f}s)")

    print("\nPhase 4: Computing recommendations using CPU (baseline reference).")
    cpu_top_indices, cpu_top_scores, cpu_diagonal, cpu_duration = compute_similarities_cpu_baseline(
        product_basket_matrix_cpu, top_k=recommendations_count)
    print(f" CPU execution duration: {cpu_duration:.2f}s")

    # Select reference product for demonstration
    if config.anchor_by == "most_popular":
        reference_product = int(np.argmax(cpu_diagonal))
    elif config.anchor_by == "random":
        reference_product = int(np.random.default_rng(1).integers(0, product_basket_matrix_cpu.shape[0]))
    else:
        if config.anchor_id is None:
            raise ValueError("Must specify anchor_id when using anchor_by='id'")
        reference_product = config.anchor_id

    reference_stock_code = product_metadata[reference_product]['stock_code']
    reference_description = product_metadata[reference_product]['description']

    print(f"\nReference product: index={reference_product}; "
          f"stock_code={reference_stock_code}; "
          f"description={reference_description}; "
          f"basket_occurrences={int(cpu_diagonal[reference_product])}")

    # Generate recommendation reports
    gpu_recommendations = generate_recommendation_report(reference_product, gpu_top_indices, gpu_top_scores,
                                                        product_basket_matrix_cpu, gpu_diagonal,
                                                        product_metadata, total_baskets)
    cpu_recommendations = generate_recommendation_report(reference_product, cpu_top_indices, cpu_top_scores,
                                                        product_basket_matrix_cpu, cpu_diagonal,
                                                        product_metadata, total_baskets)

    print("\nRecommendations (GPU-generated):")
    print(gpu_recommendations.to_string(index=False))

    print("\nRecommendations (CPU-generated):")
    print(cpu_recommendations.to_string(index=False))

    # Display sample recommendations for top products
    try:
        top_two_products = list(np.argsort(-cpu_diagonal)[:2])
        print(f"\n\n=== Sample recommendations for top 2 products (indices): {top_two_products} ===")
        for sample_product in top_two_products:
            sample_stock = product_metadata[sample_product]['stock_code']
            sample_desc = product_metadata[sample_product]['description']
            print(f"\nSample Product {sample_product} (stock_code={sample_stock}, description={sample_desc}, baskets={int(cpu_diagonal[sample_product])})")
            sample_gpu_recs = generate_recommendation_report(sample_product, gpu_top_indices, gpu_top_scores,
                                                            product_basket_matrix_cpu, gpu_diagonal,
                                                            product_metadata, total_baskets)
            sample_cpu_recs = generate_recommendation_report(sample_product, cpu_top_indices, cpu_top_scores,
                                                            product_basket_matrix_cpu, cpu_diagonal,
                                                            product_metadata, total_baskets)
            print("\nGPU recommendations (top 5):")
            print(sample_gpu_recs.head(5).to_string(index=False))
            print("\nCPU recommendations (top 5):")
            print(sample_cpu_recs.head(5).to_string(index=False))
    except Exception as error:
        print("Unable to generate sample recommendations:", error)

    # Export results
    export_recommendation_tables(reference_product, gpu_recommendations, cpu_recommendations,
                                output_prefix=f"product_{reference_product}_recommendations")
    if config.save_all:
        print("\nExporting complete recommendation datasets. This may take time for large catalogs.")
        # CPU results export
        all_cpu_rows = []
        for prod_idx in range(len(cpu_top_indices)):
            indices = cpu_top_indices[prod_idx]
            scores = cpu_top_scores[prod_idx]
            for rank_pos, (related_idx, score_val) in enumerate(zip(indices, scores), start=1):
                all_cpu_rows.append({
                    "item_index": prod_idx,
                    "stock_code_item": product_metadata[prod_idx]["stock_code"],
                    "item_description": product_metadata[prod_idx]["description"],
                    "rank": rank_pos,
                    "other_index": int(related_idx),
                    "other_stock_code": product_metadata[int(related_idx)]["stock_code"],
                    "other_description": product_metadata[int(related_idx)]["description"],
                    "similarity": float(score_val),
                })
        pd.DataFrame(all_cpu_rows).to_csv("recommendations_complete_cpu.csv", index=False)
        # GPU results export
        all_gpu_rows = []
        for prod_idx in range(len(gpu_top_indices)):
            indices = gpu_top_indices[prod_idx]
            scores = gpu_top_scores[prod_idx]
            for rank_pos, (related_idx, score_val) in enumerate(zip(indices, scores), start=1):
                all_gpu_rows.append({
                    "item_index": prod_idx,
                    "stock_code_item": product_metadata[prod_idx]["stock_code"],
                    "item_description": product_metadata[prod_idx]["description"],
                    "rank": rank_pos,
                    "other_index": int(related_idx),
                    "other_stock_code": product_metadata[int(related_idx)]["stock_code"],
                    "other_description": product_metadata[int(related_idx)]["description"],
                    "similarity": float(score_val),
                })
        pd.DataFrame(all_gpu_rows).to_csv("recommendations_complete_gpu.csv", index=False)
        print("Exported recommendations_complete_cpu.csv and recommendations_complete_gpu.csv")

    # Performance summary
    print("\nPERFORMANCE SUMMARY:")
    summary_data = pd.DataFrame([
        {"computation_method": "CPU (SciPy) row-wise", "execution_time_sec": round(cpu_duration, 3)},
        {"computation_method": "GPU (CuPy) data transfer", "execution_time_sec": round(transfer_duration, 3)},
        {"computation_method": "GPU (CuPy) computation", "execution_time_sec": round(gpu_compute_duration, 3)},
        {"computation_method": "GPU (CuPy) combined", "execution_time_sec": round(gpu_total_duration, 3)}
    ])
    speedup_factor = (cpu_duration / gpu_total_duration) if (gpu_total_duration > 0) else float('nan')
    summary_display = summary_data.copy()
    if not np.isnan(speedup_factor):
        summary_display.loc[len(summary_display)] = {
            "computation_method": "Speedup (CPU/GPU)",
            "execution_time_sec": round(speedup_factor, 3)
        }
    print(summary_display.to_string(index=False))

    # Visualization
    if config.inline_plot:
        try:
            visualize_performance_comparison(cpu_duration, transfer_duration, gpu_compute_duration)
        except Exception as error:
            print("Warning: inline visualization failed:", error)
    else:
        try:
            from matplotlib import pyplot as plot_module
            gpu_combined = transfer_duration + gpu_compute_duration
            plot_data = pd.DataFrame([
                {"computation_method": "CPU (SciPy) row-wise", "execution_time_sec": cpu_duration},
                {"computation_method": "GPU (CuPy) data transfer", "execution_time_sec": transfer_duration},
                {"computation_method": "GPU (CuPy) computation", "execution_time_sec": gpu_compute_duration},
                {"computation_method": "GPU (CuPy) combined", "execution_time_sec": gpu_combined}
            ])
            plot_module.figure(figsize=(8, 4))
            bar_plot = plot_module.bar(plot_data["computation_method"], plot_data["execution_time_sec"])
            plot_module.ylabel("Execution Time (seconds)")
            plot_module.title("Performance Comparison: CPU vs GPU")
            for bar in bar_plot:
                height = bar.get_height()
                plot_module.text(bar.get_x() + bar.get_width()/2, height + 0.5,
                               f"{height:.2f}s", ha='center', va='bottom', fontsize=9)
            plot_module.tight_layout()
            plot_module.savefig("performance_analysis.png", dpi=150)
            plot_module.close()
            print("Performance visualization saved: performance_analysis.png")
        except Exception as error:
            print("Warning: visualization generation failed:", error)

    display_implementation_summary()
    print("\nExecution completed successfully.")

if __name__ == '__main__':
    if hasattr(sys, 'argv') and len(sys.argv) > 0 and \
       (sys.argv[0].endswith('ipykernel_launcher.py') or sys.argv[0].endswith('colab_kernel_launcher.py')):
        main(argv=[])
    else:
        main(argv=sys.argv[1:])

Phase 1: Loading and preprocessing transaction dataset.
 Dataset loaded: products=3922, baskets=19960, duration=2.73s

Phase 2: Transferring sparse matrix to GPU memory.
 Transfer completed: 0.02s

Phase 3: Computing recommendations using GPU (batched processing).
[GPU processing] batch 0-63 complete (size=64), time=0.02s
[GPU processing] batch 64-127 complete (size=64), time=0.03s
[GPU processing] batch 128-191 complete (size=64), time=0.04s
[GPU processing] batch 192-255 complete (size=64), time=0.06s
[GPU processing] batch 256-319 complete (size=64), time=0.07s
[GPU processing] batch 320-383 complete (size=64), time=0.09s
[GPU processing] batch 384-447 complete (size=64), time=0.10s
[GPU processing] batch 448-511 complete (size=64), time=0.11s
[GPU processing] batch 512-575 complete (size=64), time=0.13s
[GPU processing] batch 576-639 complete (size=64), time=0.14s
[GPU processing] batch 640-703 complete (size=64), time=0.15s
[GPU processing] batch 704-767 complete (size=64), time=0